In [1]:
import pandas as pd
import numpy as np
import os
import string
import sys
import re
import datetime as dt
import import_ipynb
#os정렬
import natsort
from tqdm import tqdm
# from standard_func import *
# import standard_func_py as sf
# now = dt.datetime.now().strftime('%y%m%d_%H%M%S')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 진행 여부 함수
def yes_or_no(question):
    while "the answer is invalid":
        reply = str(input(question+' (y/n): ')).lower().strip()
        if reply[0] == 'y':
            return True
        if reply[0] == 'n':
            return False
        else:
            print("실패: 'y' 혹은 'n' 를 입력해 주세요")

In [9]:
# 불러오기
path = 'C:/Users/namdd/OneDrive/Data/Standardization/dict'
path_dir = 'C:/Users/namdd/OneDrive/Data/Standardization/dict/전체 내려받기_표준국어대사전_xls_20221101'
file_list = os.listdir(path_dir)
# window 정렬기준 사용
file_list = natsort.natsorted(file_list)

In [3]:
# 파일명(xlsx) 제거
file_list_Nxls = []
for i in file_list:
    file_list_Nxls.append(i.split('.')[0])
file_list_Nxls

['1047090_30000',
 '1047090_60000',
 '1047090_90000',
 '1047090_120000',
 '1047090_150000',
 '1047090_180000',
 '1047090_210000',
 '1047090_240000',
 '1047090_270000',
 '1047090_300000',
 '1047090_330000',
 '1047090_360000',
 '1047090_390000',
 '1047090_420000',
 '1047090_434216']

In [4]:
# 모든 파일 읽어오기
for i in tqdm(file_list_Nxls):
    globals()["dict_{}".format(i)] = pd.read_excel(path_dir + "/" + i + ".xls")

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:32<00:00,  2.19s/it]


In [7]:
# 모든 파일 병합
dict_total = pd.DataFrame()
for i in tqdm(file_list_Nxls):
    dict_total = pd.concat([dict_total, globals()["dict_{}".format(i)]], axis = 0, ignore_index = True)
dict_smpl_mrg = dict_total.copy()    
print(dict_total.shape)
dict_total.head(2)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:01<00:00, 13.82it/s]


(434216, 26)


,어휘,구성 단위,고유어 여부,원어,원어·어종,어원,주표제어,부표제어,발음,활용,검색용 이형태,품사,공통 문형,의미 문형,공통 문법,의미 문법,뜻풀이,용례,범주,전문 분야,속담,관용구,대역어,생물 분류군 정보,멀티미디어,관련 어휘
0,가(01),단어,고유어,NaN,NaN,＜＜＜용가＞,NaN,NaN,[가ː],NaN,NaN,「명사」\n,NaN,「1」\n「2」\n「3」\n「4」\n,NaN,「1」\n「2」\n「3」\n「4」((일부 명사 뒤에 붙어))\n,「1」경계에 가까운 바깥쪽 부분.\n「2」어떤 중심 되는 곳에서 가까운 부분.\n「...,「3」참기름을 따를 때 가에 흘리지 않도록 조심해라.\n「4」강가.\n「4」냇가.\...,「1」일반어\n「2」일반어\n「3」일반어\n「4」일반어\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,가(02),단어,고유어,NaN,NaN,NaN,NaN,NaN,[가],NaN,NaN,「명사」\n,NaN,\n,NaN,\n,"서양 음악의 칠음 체계에서, 여섯 번째 음이름.\n",NaN,일반어\n,『음악』\n,NaN,NaN,NaN,NaN,NaN,"의미\n동의어(2) : 아08(A), 에이05(A / a)\n\n"


In [10]:
dict_smpl_mrg.to_excel(path + '/dict_smpl_mrg/' + 'dict_smpl_mrg.xlsx')

In [11]:
def emoti_chk(data, col, rglr_exp, *opt):
    
    # 이력 보기 옴션
    opt = ''.join(opt)
    if opt == 'view':
        stwd_total = data

        # 한글명에 영문 소문자 탐색
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in tqdm(range(len(stwd_total))):

            # 조건에 맞는 단어만 체크
            cor_kor = re.findall(rglr_exp, stwd_total.loc[i, col])
            # 조건에 맞는 단어 수 체크
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, col]))
            
            # 이력 저장
            if bool(chk_kor) == False:
                print("----------------------------- 수정 ----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, col] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 한글 외 변경 수정
            stwd_total.loc[i, 'kor_ec'] = re.sub(rglr_exp, '', stwd_total.loc[i, col])        

            # 한글 외 자동 삭제
            stwd_total.loc[i, col] = ''.join(cor_kor)

        # 변경 이력 저장
        stwd_total['kor_cor_hist'] = ""
        stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("한글 조합어 외 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')
    else:
        
        # 이력 보지 않기 옴션(default)
        stwd_total = data

        # 한글명에 영문 소문자 탐색
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in tqdm(range(len(stwd_total))):

            # 조건에 맞는 단어만 체크
            cor_kor = re.findall(rglr_exp, stwd_total.loc[i, col])
            # 조건에 맞는 단어 수 체크
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, col]))
            
            # 이력 저장
            if bool(chk_kor) == False:
                kor_chk_list.append(i)

            # 한글 외 변경 수정
            stwd_total.loc[i, 'kor_ec'] = re.sub(rglr_exp, '', stwd_total.loc[i, col])        

            # 한글 외 자동 삭제
            stwd_total.loc[i, col] = ''.join(cor_kor)

        # 변경 이력 저장
        stwd_total['kor_cor_hist'] = ""
        stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("한글 조합어 외 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

        

In [12]:
# 모든 품사 특수기호 제거
emoti_chk(dict_total, '품사', "[가-힣+]")

100%|████████████████████████████████████████████████████████████████████████| 434216/434216 [01:34<00:00, 4598.51it/s]


====================================한글명 검토====================================|
한글 조합어 외 제거: 434216
[자동 처리 완료하였습니다]


In [13]:
pumsa_unique = dict_total['품사'].unique()
pumsa_unique

array(['명사', '품사없음', '조사', '접사', '동사', '형용사', '명사부사', '부사', '구', '명사관형사',
       '부사명사', '동사보조동사', '부사감탄사', '의존명사', '동사형용사', '감탄사', '명사의존명사', '관형사',
       '수사관형사', '형용사동사', '부사조사', '의존명사대명사', '대명사', '어미', '대명사감탄사',
       '명사감탄사', '부사품사없음', '명사대명사', '대명사관형사', '관형사품사없음', '감탄사품사없음',
       '감탄사명사', '명사수사', '명사품사없음', '명사수사관형사', '관형사감탄사', '대명사명사', '수사',
       '수사관형사명사', '의존명사명사', '어미품사없음', '보조형용사', '보조동사', '의존명사조사',
       '명사수사관형사부사', '의존명사수사관형사', '동사형용사보조동사보조형용사', '명사부사감탄사',
       '동사보조동사보조형용사', '품사없음명사', '명사관형사부사', '명사의존명사대명사', '보조동사보조형용사',
       '감탄사부사', '부사감탄사품사없음', '대명사부사', '동사형용사보조동사', '동사보조형용사', '수사관형사명사부사',
       '수사명사'], dtype=object)

In [14]:
# 모든 어휘 특수기호 제거
emoti_chk(dict_total, '어휘', "[가-힣+]")

100%|████████████████████████████████████████████████████████████████████████| 434216/434216 [02:52<00:00, 2510.76it/s]


====================================한글명 검토====================================|
한글 조합어 외 제거: 372956
[자동 처리 완료하였습니다]


In [15]:
eohee_unique = dict_total['어휘'].unique()
eohee_unique

array(['가', '가가', '가가대소', ..., '딘다', '받침체언', '종성체언'], dtype=object)

In [16]:
# 품사 기준 50개 데이터 샘플
dict_clssf = dict_total.columns
pumsa_examp = pd.DataFrame()
select_cnt = 50
for i, j in tqdm(zip(range(len(pumsa_unique)), pumsa_unique)):

    row_cnt = len(dict_total.loc[dict_total['품사'] == j, '어휘'])
    globals()["pumsa_unique_{}".format(i)] = pd.DataFrame(
        dict_total.loc[dict_total['품사'] == j, '어휘'].head(select_cnt).reset_index(drop=True))
    globals()["pumsa_unique_{}".format(i)].columns = [j]
    globals()["pumsa_unique_{}".format(i)].loc[select_cnt+1,j] = row_cnt
    pumsa_examp = pd.concat([pumsa_examp, globals()["pumsa_unique_{}".format(i)]], axis = 1)
print(pumsa_examp.shape)
pumsa_examp.tail(5)

60it [00:06,  9.29it/s]

(51, 60)


,명사,품사없음,조사,접사,동사,형용사,명사부사,부사,구,명사관형사,부사명사,동사보조동사,부사감탄사,의존명사,동사형용사,감탄사,명사의존명사,관형사,수사관형사,형용사동사,부사조사,의존명사대명사,대명사,어미,대명사감탄사,명사감탄사,부사품사없음,명사대명사,대명사관형사,관형사품사없음,감탄사품사없음,감탄사명사,명사수사,명사품사없음,명사수사관형사,관형사감탄사,대명사명사,수사,수사관형사명사,의존명사명사,어미품사없음,보조형용사,보조동사,의존명사조사,명사수사관형사부사,의존명사수사관형사,동사형용사보조동사보조형용사,명사부사감탄사,동사보조동사보조형용사,품사없음명사,명사관형사부사,명사의존명사대명사,보조동사보조형용사,감탄사부사,부사감탄사품사없음,대명사부사,동사형용사보조동사,동사보조형용사,수사관형사명사부사,수사명사
46,가거,가견,란,관,가납되다,가란하다,내일모레,가뜩이나,가닥이잡히다,거시적,NaN,NaN,NaN,국제옴,기연미연하다,둥개둥개,바람,무슨,삼,NaN,NaN,NaN,그분,기로,NaN,어깨총,NaN,소사,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,려거든,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,가거,가경,랑,관,가납되다,가랄하다,낼,가뜩한데,가도오도못하다,거인적,NaN,NaN,NaN,국제옹스트롬,기우뚱기우뚱하다,둥둥,바이트,뭇,삼십,NaN,NaN,NaN,그이,기로니,NaN,어부바,NaN,소인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,련,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,가거지지,가계경제,로,관,가납하다,가랑가랑하다,낼모레,가뜩히,가락을떼다,거족적,NaN,NaN,NaN,국제와트,기우뚱하다,둬둬,바퀴,뭔,서너째,NaN,NaN,NaN,그자,기로서,NaN,어비,NaN,소자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,렵니까,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,가거처,가계미가,로부터,광,가납하다,가량가량하다,냠냠,가뜬가뜬,가락이나다,거족적,NaN,NaN,NaN,군데,기웃기웃하다,둬둬둬,박,바른,서른,NaN,NaN,NaN,그쪽,기로서니,NaN,엎드려뻗쳐,NaN,소제,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,렵니다,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,266276,70520,173,533,55503,12721,300,11912,11308,1229,19,27,19,720,625,503,132,138,123,24,2,4,251,729,6,80,23,75,13,7,9,9,1,9,12,12,1,18,16,4,77,11,6,4,1,4,1,4,3,5,1,1,2,2,1,1,1,1,1,3


In [19]:
aa = pd.read_excel(path + '/dict_smpl_mrg/' + 'dict_smpl_mrg.xlsx')
aa

,Unnamed: 0,어휘,구성 단위,고유어 여부,원어,원어·어종,어원,주표제어,부표제어,발음,활용,검색용 이형태,품사,공통 문형,의미 문형,공통 문법,의미 문법,뜻풀이,용례,범주,전문 분야,속담,관용구,대역어,생물 분류군 정보,멀티미디어,관련 어휘
0,0,가(01),단어,고유어,NaN,NaN,＜＜＜용가＞,NaN,NaN,[가ː],NaN,NaN,「명사」\n,NaN,「1」\n「2」\n「3」\n「4」\n,NaN,「1」\n「2」\n「3」\n「4」((일부 명사 뒤에 붙어))\n,「1」경계에 가까운 바깥쪽 부분.\n「2」어떤 중심 되는 곳에서 가까운 부분.\n「...,「3」참기름을 따를 때 가에 흘리지 않도록 조심해라.\n「4」강가.\n「4」냇가.\...,「1」일반어\n「2」일반어\n「3」일반어\n「4」일반어\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,가(02),단어,고유어,NaN,NaN,NaN,NaN,NaN,[가],NaN,NaN,「명사」\n,NaN,\n,NaN,\n,"서양 음악의 칠음 체계에서, 여섯 번째 음이름.\n",NaN,일반어\n,『음악』\n,NaN,NaN,NaN,NaN,NaN,"의미\n동의어(2) : 아08(A), 에이05(A / a)\n\n"
2,2,가(03),단어,한자어,加,한자 加\n,NaN,NaN,가-하다,NaN,NaN,NaN,「품사 없음」\n,NaN,\n,NaN,\n,‘가하다01’의 어근.\n,NaN,일반어\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,가(04),단어,한자어,加,한자 加\n,NaN,NaN,NaN,[가],NaN,NaN,「명사」\n,NaN,\n,NaN,\n,"부여와 고구려에서, 족장이나 고관을 이르던 말. 본디 씨족이나 부족의 우두머리를 뜻...",NaN,일반어\n,『역사』\n,NaN,NaN,NaN,NaN,NaN,NaN
4,4,가(05),단어,한자어,可,한자 可\n,NaN,NaN,가-하다,[가ː],NaN,NaN,「명사」\n,NaN,「1」\n「2」\n「3」\n「4」\n,NaN,「1」\n「2」\n「3」\n「4」\n,"「1」옳거나 좋음.\n「2」회의 따위에서, 어떤 안건에 대하여 표결을 할 때 찬성하...","「1」이 사람 말도 가요, 저 사람 말도 가요 하면 도대체 어떤 사람 말을 따라야 ...",「1」일반어\n「2」일반어\n「3」일반어\n「4」일반어\n,「3」『교육』\n,NaN,NaN,NaN,NaN,NaN,"의미\n반대말(2) : 「2」 부07「2」(否), 「4」 불가01「2」(不可)\n\n"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434211,434211,ㅟ,단어,고유어,NaN,NaN,NaN,NaN,NaN,[위],NaN,NaN,「명사」\n,NaN,\n,NaN,\n,"한글 자모 ‘ㅜ’와 ‘ㅣ’를 어울러 쓴 글자. ‘위’라고 이르며, ‘귀’에서와 같이...",NaN,일반어\n,『언어』\n,NaN,NaN,NaN,NaN,NaN,NaN
434212,434212,ㅠ,단어,고유어,NaN,NaN,NaN,NaN,NaN,[유],NaN,NaN,「명사」\n,NaN,\n,NaN,\n,"한글 자모의 스물두째 글자. ‘유’라고 이르며, ‘규, 육’에서와 같이 중성으로 쓰...",NaN,일반어\n,『언어』\n,NaN,NaN,NaN,NaN,NaN,NaN
434213,434213,ㅡ,단어,고유어,NaN,NaN,NaN,NaN,NaN,[으],NaN,NaN,「명사」\n,NaN,\n,NaN,\n,"한글 자모의 스물셋째 글자. ‘으’라고 이르며, ‘그, 극’에서와 같이 중성으로 쓰...",NaN,일반어\n,『언어』\n,NaN,NaN,NaN,NaN,NaN,NaN
434214,434214,ㅢ,단어,고유어,NaN,NaN,NaN,NaN,NaN,[의],NaN,NaN,「명사」\n,NaN,\n,NaN,\n,"한글 자모 ‘ㅡ’와 ‘ㅣ’를 어울러 쓴 글자. ‘의’라고 이르며, ‘의’에서와 같이...",NaN,일반어\n,『언어』\n,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
pumsa_examp.to_excel(path + '/pumsa_examp/' + 'pumsa_examp.xlsx')

In [21]:
dict_total.to_excel(path + '/dict_total/' + 'dict_total.xlsx')

### 20만단어 초반대인데... expln 입히려면 시간 좀 걸리는게 맞고......
20만단어를 가지고 표준단어사전을 점검하는게 과연 가능할지.......

In [32]:
dict_insp =  pd.DataFrame(dict_total.loc [dict_total['품사'] == '명사', '어휘'].unique(), columns= ['어휘'])
print(dict_insp.shape)
dict_insp

(210474, 1)


,어휘
0,가
1,가가
2,가가대소
3,가가례
4,가가린
...,...
210469,자쇠
210470,자집
210471,자촉홈
210472,자턱솔홈


In [140]:
dict_total.loc[(dict_total['구성 단위'] == '단어') & (dict_total['품사'] == '명사')].shape

(266275, 29)

In [141]:
dict_total.loc[ (dict_total['품사'] == '명사')].shape
#결론:품사==명사는 구성단위=단어 조건을 포함한다 

(266276, 29)

In [33]:
dict_insp.to_excel(path + '/dict_total/' + 'dict_insp.xlsx')

In [133]:
# 1글자 이상 찾기
alist = []
blist = []
for i in tqdm(range(len(dict_expn['어휘']))):
#     if len(dict_expn.loc[i ,'어휘']) > 1:
    if len(dict_expn.iloc[i,0]) > 1:
        alist.append(i)
    elif len(dict_expn.iloc[i,0]) == 1:
        blist.append(i)
alist
blist

100%|███████████████████████████████████████████████████████████████████████| 266276/266276 [00:06<00:00, 39497.80it/s]


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 2623,
 2624,
 2625,
 2626,
 2627,
 2628,
 2629,
 2630,
 2631,
 2632,
 3191,
 3192,
 3193,
 3194,
 3195,
 3196,
 3197,
 3198,
 3199,
 3949,
 3950,
 3951,
 3952,
 3953,
 3954,
 3955,
 3956,
 3957,
 3958,
 3959,
 3960,
 3961,
 4422,
 4423,
 4424,
 4425,
 4426,
 4427,
 4428,
 4429,
 4430,
 4431,
 4432,
 4433,
 4434,
 4435,
 5417,
 5418,
 5419,
 5420,
 5579,
 5582,
 5583,
 5584,
 5643,
 5644,
 5645,
 5646,
 5647,
 5648,
 5649,
 5650,
 5651,
 5652,
 5653,
 5654,
 5655,
 5656,
 5657,
 5658,
 5659,
 6678,
 6691,
 6692,
 6693,
 6694,
 6695,
 6696,
 6697,
 8017,
 8205,
 8299,
 8300,
 8301,
 8302,
 8396,
 8397,
 8398,
 8399,
 9273,
 9274,
 9275,
 9276,
 9889,
 9890,
 10039,
 10040,
 10041,
 10526,
 10527,
 10580,
 10740,
 10741,
 10742,
 10743,
 10744,
 10950,
 10966,
 11054,
 11055,
 11056,
 11057,
 11058,
 11317,
 11318,
 11319,
 11320,
 11321,
 11693,
 11694,
 11695,
 11696,
 11697,
 11698,
 11699,
 11700,
 11701,
 11702,
 12175,
 12324,
 12502,
 12503,
 12

In [126]:
dict_expn.iloc[alist].shape

(263274, 3)

In [132]:
len(dict_expn['어휘'].unique())

210474

In [115]:
dict_expn.head(10)

,어휘,뜻풀이,용례
0,가,「1」경계에 가까운 바깥쪽 부분.\n「2」어떤 중심 되는 곳에서 가까운 부분.\n「...,「3」참기름을 따를 때 가에 흘리지 않도록 조심해라.\n「4」강가.\n「4」냇가.\...
1,가,"서양 음악의 칠음 체계에서, 여섯 번째 음이름.\n",NaN
3,가,"부여와 고구려에서, 족장이나 고관을 이르던 말. 본디 씨족이나 부족의 우두머리를 뜻...",NaN
4,가,"「1」옳거나 좋음.\n「2」회의 따위에서, 어떤 안건에 대하여 표결을 할 때 찬성하...","「1」이 사람 말도 가요, 저 사람 말도 가요 하면 도대체 어떤 사람 말을 따라야 ..."
5,가,죄인에게 씌우던 형틀. 두껍고 긴 널빤지의 한끝에 구멍을 뚫어 죄인의 목을 끼우고 ...,NaN
6,가,"예전에, 같은 호적에 들어 있는 친족 집단을 이르던 말.\n",성구는 그 대목에서 묘하게 처절해지는 버릇이 있었다. 외할머니 교하댁의 집에 대한 ...
7,가,짐승의 뿔로 만든 원시적인 악기. 신라 때 이 악기의 반주에 맞추어 추던 가무(笳舞...,NaN
8,가,제례 때에 쓰던 술잔. 청동으로 만들며 손잡이가 달린 몸통에 세 개의 다리와 두 개...,NaN
9,가,"우리나라 성(姓)의 하나. 본관은 소주(蘇州), 태안(泰安) 등 10여 본이 현존한...",\n
18,가가,"주로 편지에서, 스스로 생각하여도 우습다는 뜻으로 이르는 말.\n",NaN


In [113]:
dict_expn.iloc[9,0]

'가가'

In [112]:
len(alist)

263274

In [85]:
len(dict_expn)

266276

In [86]:
alist = []
for i in tqdm(range(len(dict_expn['어휘']))):
    if len(dict_expn.loc[i ,'어휘']) > 1:
        alist.append(i)
alist

  0%|                                                                             | 2/266276 [00:00<00:29, 9078.58it/s]


KeyError: 2

In [76]:
dict_expn.loc[: ,'어휘'].head(400)

0               가
1               가
3               가
4               가
5               가
6               가
7               가
8               가
9               가
18             가가
20             가가
21           가가대소
23            가가례
24            가가린
25           가가문전
26            가가방
27           가가성자
29           가가와현
33             가각
34            가각고
35            가각본
38             가간
39            가간사
40           가간집물
41             가감
44            가감례
45            가감법
46            가감변
47           가감부득
48            가감산
51           가감승제
52          가감승합제
53            가감역
54           가감역관
55           가감은휘
57           가감죄례
58           가감지인
60            가감판
63           가감합제
64            가감형
68             가객
69             가객
70             가갸
71            가갸글
72            가갸날
74             가거
75             가거
76             가거
77           가거지지
78            가거처
81            가건물
82            가건축
83            가검물
84             가게
85          가게거밋과
87        

In [65]:
range(len(dict_expn['어휘']))

range(0, 266276)

In [77]:
len(dict_expn.loc[18 ,'어휘'])

2

In [64]:
dict_expn.loc[0 ,'어휘']

'가'

In [63]:
alist

[]

In [62]:
dict_expn = dict_total.loc [dict_total['품사'] == '명사', ['어휘', '뜻풀이', '용례']]
print(dict_expn.shape)
dict_expn

(266276, 3)


,어휘,뜻풀이,용례
0,가,「1」경계에 가까운 바깥쪽 부분.\n「2」어떤 중심 되는 곳에서 가까운 부분.\n「...,「3」참기름을 따를 때 가에 흘리지 않도록 조심해라.\n「4」강가.\n「4」냇가.\...
1,가,"서양 음악의 칠음 체계에서, 여섯 번째 음이름.\n",NaN
3,가,"부여와 고구려에서, 족장이나 고관을 이르던 말. 본디 씨족이나 부족의 우두머리를 뜻...",NaN
4,가,"「1」옳거나 좋음.\n「2」회의 따위에서, 어떤 안건에 대하여 표결을 할 때 찬성하...","「1」이 사람 말도 가요, 저 사람 말도 가요 하면 도대체 어떤 사람 말을 따라야 ..."
5,가,죄인에게 씌우던 형틀. 두껍고 긴 널빤지의 한끝에 구멍을 뚫어 죄인의 목을 끼우고 ...,NaN
...,...,...,...
434211,,"한글 자모 ‘ㅜ’와 ‘ㅣ’를 어울러 쓴 글자. ‘위’라고 이르며, ‘귀’에서와 같이...",NaN
434212,,"한글 자모의 스물두째 글자. ‘유’라고 이르며, ‘규, 육’에서와 같이 중성으로 쓰...",NaN
434213,,"한글 자모의 스물셋째 글자. ‘으’라고 이르며, ‘그, 극’에서와 같이 중성으로 쓰...",NaN
434214,,"한글 자모 ‘ㅡ’와 ‘ㅣ’를 어울러 쓴 글자. ‘의’라고 이르며, ‘의’에서와 같이...",NaN


In [34]:
dict_expn.to_excel(path + '/dict_total/' + 'dict_expn.xlsx')

In [35]:
test = pd.read_excel(path + '/dict_total/' + 'dict_expn.xlsx')
test

,Unnamed: 0,어휘,뜻풀이,용례
0,0,가,「1」경계에 가까운 바깥쪽 부분.\n「2」어떤 중심 되는 곳에서 가까운 부분.\n「...,「3」참기름을 따를 때 가에 흘리지 않도록 조심해라.\n「4」강가.\n「4」냇가.\...
1,1,가,"서양 음악의 칠음 체계에서, 여섯 번째 음이름.\n",NaN
2,3,가,"부여와 고구려에서, 족장이나 고관을 이르던 말. 본디 씨족이나 부족의 우두머리를 뜻...",NaN
3,4,가,"「1」옳거나 좋음.\n「2」회의 따위에서, 어떤 안건에 대하여 표결을 할 때 찬성하...","「1」이 사람 말도 가요, 저 사람 말도 가요 하면 도대체 어떤 사람 말을 따라야 ..."
4,5,가,죄인에게 씌우던 형틀. 두껍고 긴 널빤지의 한끝에 구멍을 뚫어 죄인의 목을 끼우고 ...,NaN
...,...,...,...,...
266271,434211,NaN,"한글 자모 ‘ㅜ’와 ‘ㅣ’를 어울러 쓴 글자. ‘위’라고 이르며, ‘귀’에서와 같이...",NaN
266272,434212,NaN,"한글 자모의 스물두째 글자. ‘유’라고 이르며, ‘규, 육’에서와 같이 중성으로 쓰...",NaN
266273,434213,NaN,"한글 자모의 스물셋째 글자. ‘으’라고 이르며, ‘그, 극’에서와 같이 중성으로 쓰...",NaN
266274,434214,NaN,"한글 자모 ‘ㅡ’와 ‘ㅣ’를 어울러 쓴 글자. ‘의’라고 이르며, ‘의’에서와 같이...",NaN


In [53]:
for i in tqdm(dict_insp['어휘']):
    for j in tqdm(dict_expn['어휘']):
        i==j
        print(i)
        print(j)
#     (dict_insp.iloc[i])
#     print(i)

  0%|▏                                                                          | 877/266276 [00:00<00:36, 7338.40it/s]

가
가
가
가
가
가
가
가
가
가
가
가
가
가
가
가
가
가
가
가가
가
가가
가
가가대소
가
가가례
가
가가린
가
가가문전
가
가가방
가
가가성자
가
가가와현
가
가각
가
가각고
가
가각본
가
가간
가
가간사
가
가간집물
가
가감
가
가감례
가
가감법
가
가감변
가
가감부득
가
가감산
가
가감승제
가
가감승합제
가
가감역
가
가감역관
가
가감은휘
가
가감죄례
가
가감지인
가
가감판
가
가감합제
가
가감형
가
가객
가
가객
가
가갸
가
가갸글
가
가갸날
가
가거
가
가거
가
가거
가
가거지지
가
가거처
가
가건물
가
가건축
가
가검물
가
가게
가
가게거밋과
가
가게내기
가
가게쟁이
가
가게채
가
가겟방
가
가겟세
가
가겟집
가
가격
가
가격
가
가격
가
가격대
가
가격소구
가
가격시점
가
가격제
가
가격표
가
가결
가
가결
가
가결
가
가결안
가
가결의
가
가결전
가
가경
가
가경
가
가경
가
가경
가
가경
가
가경
가
가경
가
가경일
가
가경자
가
가경자
가
가경전
가
가경절
가
가경지
가
가계
가
가계
가
가계
가
가계
가
가계
가
가계
가
가계
가
가계
가
가계
가
가계
가
가계도
가
가계례
가
가계부
가
가계부기
가
가계비
가
가계약
가
가계정
가
가계주
가
가계표
가
가고
가
가고
가
가고
가
가고건
가
가고문적
가
가고문헌
가
가고시마
가
가고시마현
가
가고처
가
가고한
가
가곡
가
가곡
가
가곡보
가
가곡선
가
가곡원류
가
가곡집
가
가골
가
가공
가
가공
가
가공
가
가공
가
가공가소
가
가공간
가
가공망상
가
가공물
가
가공법
가
가공법
가
가공비
가
가공사
가
가공선
가
가공식품
가
가공업
가
가공영사
가
가공유
가
가공육
가
가공인물
가
가공절목
가
가공주
가
가공지
가
가공창고
가
가공치
가
가공품
가
가과
가
가과
가
가과
가
가곽
가
가관
가
가관
가
가관
가
가관
가
가관
가
가관
가
가관노
가
가관속
가
가관절
가
가관절증
가
가광
가
가교
가
가교
가
가교
가
가교
가
가교
가
가교
가
가교마
가
가교봉도
가
가교사
가
가교실


  1%|▋                                                                         | 2370/266276 [00:00<00:40, 6519.20it/s]


가속도계
가
가속도병
가
가속화
가
가손
가
가손
가
가솔
가
가솔근수
가
가솔린
가
가솔린차
가
가솔린펌프
가
가솔송
가
가송
가
가송장
가
가쇄
가
가쇄
가
가수
가
가수
가
가수
가
가수
가
가수
가
가수
가
가수
가
가수
가
가수
가
가수
가
가수
가
가수
가
가수금
가
가수라놀린
가
가수부
가
가수성장
가
가수알바람
가
가수요
가
가수요자
가
가수용소
가
가숙
가
가숙
가
가숙본
가
가술
가
가숭어
가
가스
가
가스경유
가
가스계량기
가
가스관
가
가스기름
가
가스난로
가
가스냉장고
가
가스대
가
가스도관
가
가스등
가
가스라기
가
가스라이터
가
가스라이트지
가
가스랑이
가
가스램프
가
가스레인지
가
가스로
가
가스맨틀
가
가스미가우라호
가
가스버너
가
가스병
가
가스보일러
가
가스봄베
가
가스블랙
가
가스사
가
가스새
가
가스스토브
가
가스실
가
가스실
가
가스액
가
가스양계
가
가스오븐레인지
가
가스욕
가
가스유
가
가스잡이
가
가스전
가
가스전
가
가스정
가
가스주머니
가
가스지지미
가
가스직
가
가스총
가
가스층
가
가스코뉴
가
가스코뉴만
가
가스탄
가
가스탱크
가
가스테리아
가
가스통
가
가스트린
가
가스파초
가
가스펠
가
가스풍로
가
가스화
가
가스화제
가
가스히터
가
가슬령
가
가슴
가
가슴걸이
가
가슴골
가
가슴관
가
가슴깊이
가
가슴너비
가
가슴놀이
가
가슴다리
가
가슴동
가
가슴둘레
가
가슴등뼈
가
가슴마디
가
가슴막
가
가슴막안
가
가슴막액
가
가슴막염
가
가슴만
가
가슴바대
가
가슴벽
가
가슴뼈
가
가슴살
가
가슴샘
가
가슴샘종
가
가슴소리
가
가슴속
가
가슴숨
가
가슴쓰림
가
가슴안
가
가슴안보개
가
가슴앓이
가
가슴앓잇병
가
가슴우리
가
가슴점박이먼지벌레
가
가슴지느러미
가
가슴통
가
가슴팍
가
가슴판
가
가슴패기
가
가습
가
가습기
가
가승
가
가승
가
가승
가
가승
가
가승
가
가승
가
가승미
가
가승전색
가
가승지
가
가시
가
가시
가
가시
가
가시
가
가시
가
가시
가
가시
가
가시가자미
가
가시각
가
가시개미
가
가


  1%|█                                                                         | 3864/266276 [00:00<00:38, 6778.98it/s]

...


 33%|████████████████████████▍                                                | 89085/266276 [00:13<00:30, 5747.47it/s]

가
방편보리
가
방편유여토
가
방편토
가
방편화토
가
방평
가
방폐
가
방폐
가
방포
가
방포
가
방포
가
방포성
가
방포연
가
방폭
가
방폭등
가
방폭형
가
방풍
가
방풍
가
방풍나물
가
방풍널
가
방풍림
가
방풍막
가
방풍벽
가
방풍원
가
방풍장
가
방풍재
가
방풍죽
가
방풍창
가
방풍채
가
방풍통성산
가
방풍판
가
방하
가
방학
가
방한
가
방한
가
방한
가
방한
가
방한구
가
방한력
가
방한모
가
방한모자
가
방한벽
가
방한복
가
방한의
가
방한화
가
방함
가
방함록
가
방합례
가
방해
가
방해
가
방해꾼
가
방해물
가
방해석
가
방해자
가
방해죄
가
방해초
가
방해해
가
방행
가
방행
가
방향
가
방향
가
방향
가
방향각
가
방향기
가
방향반
가
방향부
가
방향비
가
방향산
가
방향성
가
방향성
가
방향수
가
방향수제
가
방향욕
가
방향유
가
방향제
가
방향족화
가
방향치
가
방향키
가
방향타
가
방향틀
가
방향표
가
방헌
가
방험병
가
방현령
가
방현재
가
방혈
가
방형
가
방형
가
방형구법
가
방형분
가
방형파
가
방호
가
방호
가
방호벽
가
방호별감
가
방호복
가
방호소
가
방호의복
가
방호책
가
방혼
가
방화
가
방화
가
방화
가
방화
가
방화
가
방화
가
방화
가
방화광
가
방화대
가
방화대
가
방화림
가
방화문
가
방화범
가
방화벽
가
방화복
가
방화사
가
방화선
가
방화수
가
방화수
가
방화수류정
가
방화자
가
방화전
가
방화전
가
방화제
가
방화죄
가
방화포
가
방환
가
방환
가
방환
가
방환
가
방황
가
방황
가
방황실조
가
방회
가
방회
가
방효
가
방효유
가
방훈
가
방훼
가
방휼
가
방휼지세
가
방휼지쟁
가
방흥
가
방희
가
방희편
가
밭
가
밭가리
가
밭각
가
밭갈이
가
밭갈이소리
가
밭갈이철
가
밭걷이
가
밭걸이
가
밭고누
가
밭고랑
가
밭곡
가
밭곡식
가
밭골
가
밭관수
가
밭구실
가
밭귀
가
밭귀때기
가
밭길
가
밭김
가
밭날갈이
가
밭농사
가
밭다리
가
밭단골
가
밭담
가
밭당
가
밭도랑
가
밭도지
가
밭돌
가
밭두둑
가


  0%|                                                                                       | 0/210474 [00:13<?, ?it/s]



배종인
가
배좌
가
배주
가
배주
가
배주
가
배주
가
배주기
가
배준
가
배중
가
배중론
가
배중률
가
배중손
가
배증
가
배지
가
배지
가
배지
가
배지
가
배지
가
배지
가
배지기
가
배지느러미
가
배진
가
배진
가
배진
가
배진
가
배진
가
배진동
가
배질
가
배징
가
배짱
가
배짱파
가
배차
가
배차
가
배차
가
배차계
가
배차기
가
배차원
가
배착걸음
가
배찰
가
배참
가
배참
가
배참
가
배창
가
배창
가
배창
가
배창자
가
배채기
가
배책
가
배척
가
배척
가
배척
가
배척
가
배척자
가
배천
가
배천곡
가
배천지일월
가
배첩
가
배첩장
가
배청
가
배초향
가
배총
가
배추
가
배추
가
배추김치
가
배추꼬랑이
가
배추꼬랑잇국
가
배추꽃
가
배추나비고치벌
가
배추명충나방
가
배추밭
가
배추벌레
가
배추벌레살이금좀벌
가
배추속대
가
배추속대쌈
가
배추속대찜
가
배추속댓국
가
배추순나방
가
배추쌈
가
배추씨기름
가
배추씨장사
가
배추장다리
가
배추저냐
가
배추절임
가
배추좀나방
가
배추찜
가
배추통
가
배추흰나비
가
배축
가
배출
가
배출
가
배출
가
배출관
가
배출구
가
배출기
가
배출판
가
배출형
가
배춤
가
배춧국
가
배춧속
가
배춧잎
가
배치
가
배치
가
배치
가
배치기
가
배치기
가
배치도
가
배치성
가
배치전환
가
배치지
가
배코
가
배코질
가
배코칼
가
배콧자리
가
배쾨
가
배타
가
배타성
가
배타심
가
배타율
가
배타주의
가
배탁
가
배탁성
가
배탄기
가
배탈
가
배탈
가
배태
가
배태
가
배태기
가
배태기
가
배태법
가
배태성
가
배탱이
가
배터리
가
배터리
가
배턴
가
배턴루지
가
배털
가
배털보파리매
가
배토
가
배토
가
배통
가
배통
가
배퉁이
가
배트
가
배트
가
배튼
가
배튼
가
배틀걸음
가
배틀법
가
배틀재킷
가
배틱
가
배팅
가
배팅오더
가
배판
가
배판
가
배판
가
배편
가
배포
가
배포
가
배포
가
배포
가
배포망
가
배포춤
가
배폭
가
배표
가
배표
가
배품
가
배풍
가
배풍
가
배풍관
가
배풍기
가
배풍등
가
배풍

KeyboardInterrupt: 

In [44]:
dict_insp['어휘']

0            가
1           가가
2         가가대소
3          가가례
4          가가린
          ... 
210469      자쇠
210470      자집
210471     자촉홈
210472    자턱솔홈
210473      자홈
Name: 어휘, Length: 210474, dtype: object

In [ ]:
# 테스트를 위한 검색 1

inp = '관형사품사없음'
inp = '대명사명사'
dict_total.loc[dict_total['품사'] == inp]

In [ ]:
# 테스트를 위한 검색 2

inp = '가경'
dict_total.loc[dict_total['어휘'] == inp]

In [ ]:
print(dict_total.shape)
dict_total.tail()

In [ ]:
print(stwd_total.shape)
stwd_total.tail()

In [ ]:
# 데이터 검토
ans = yes_or_no("데이터 검토를 실행 하시겠습니까?")
if ans == True:
    emoti_chk(dict_total, 'kor')
    emoti_chk('eng')
elif ans == False:
    print("계속 진행하겠습니다")

In [ ]:
# 데이터 검토
ans = yes_or_no("데이터 검토를 실행 하시겠습니까?")
if ans == True:
    emoti_chk(dict_total, 'kor')
    emoti_chk('eng')
elif ans == False:
    print("계속 진행하겠습니다")

In [ ]:
def emoti_chk(data, var):
    stwd_total = data.copy()
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(stwd_total)):

        #     특수기호 제거
        #     stwd_total.loc[i, 'stwd_kor'] =  stwd_total.loc[i, 'stwd_kor'].translate(str.maketrans('', '', string.punctuation))

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(stwd_total.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'kor_lower'] = stwd_total.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_kor'] = stwd_total.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', stwd_total.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            stwd_total.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', stwd_total.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            stwd_total.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        stwd_total['kor_cor_hist'] = ""
        stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

    elif var == 'eng':
        # 영문명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_eng = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['eng_lower'] = ''
        stwd_total['eng_ec'] = ''
        eng_chk_list = []
        for i in range(len(stwd_total)):

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_eng'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_eng = pd.concat([df_lower_eng, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(stwd_total.loc[i, 'stwd_eng'].upper()))
                eng_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'eng_lower'] = stwd_total.loc[i, 'stwd_eng']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_eng'] = stwd_total.loc[i, 'stwd_eng'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_eng = re.findall('[A-Z0-9+]', stwd_total.loc[i, 'stwd_eng'])
            chk_eng = len(cor_eng) == len(list(stwd_total.loc[i, 'stwd_eng']))

            if bool(chk_eng) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(''.join(cor_eng)))
                eng_chk_list.append(i)

            # 영문대문자, 숫자 외 변경 이력 저장
            stwd_total.loc[i, 'eng_ec'] = re.sub('[A-Z0-9+]', '', stwd_total.loc[i, 'stwd_eng'])        

            # 영문대문자, 숫자 외 자동 삭제
            stwd_total.loc[i, 'stwd_eng'] = ''.join(cor_eng)

        # 변경 이력 저장
        stwd_total['eng_cor_hist'] = ''
        stwd_total['eng_cor_hist'] = stwd_total['eng_lower'] + stwd_total['eng_ec']

        # 적용 후 결과값
        print("====================================영문명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_eng)) + '개')
        stwd_total_eng_chk_1 = stwd_total.iloc[eng_chk_list]
        print("나머지 제거: "+ str(len(eng_chk_list)))
        print('[자동 처리 완료하였습니다]')
        
    else:
        print("잘못된 입력입니다( 'kor' / 'eng' )")


In [ ]:
dict_total

In [ ]:
for i in tqdm(file_list_Nxls):

In [ ]:
for i in tqdm(zip(range(len(file_list)), file_list)):
    globals()["dict_{}".format(i)] = pd.read_excel(path_dir + "/" + j)

In [ ]:
for i, j in tqdm(zip(range(len(file_list)), file_list)):
    globals()["dict_{}".format(i)] = pd.read_excel(path_dir + "/" + j)

In [ ]:
import pstuil
p = psutil.process()
p.memory_info()

In [ ]:
from tqdm import tqdm

In [ ]:
'1047090_90000.xls'

In [ ]:
dict_1.shape